In [2]:
import open3d as o3d
import os
import re

# Intrinsic parameters (can be replaced with real camera values)
width, height = 640, 480
fx, fy = 525.0, 525.0
cx, cy = 319.5, 239.5
intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

# Set folder paths
rgb_folder = "/media/amiyaun/New Volume/rugved/task8/rgb"
depth_folder = "/media/amiyaun/New Volume/rugved/task8/depth"

# Helper function to extract index from filename
def get_index(file_name):
    match = re.search(r"desk_2_(\d+)", file_name)
    return match.group(1) if match else None

# Build dictionaries of file paths indexed by ID
rgb_files = {
    get_index(f): os.path.join(rgb_folder, f)
    for f in os.listdir(rgb_folder)
    if f.endswith(".png")
}
depth_files = {
    get_index(f): os.path.join(depth_folder, f)
    for f in os.listdir(depth_folder)
    if f.endswith(".png")
}

# Find matching indices
common_indices = sorted(set(rgb_files.keys()) & set(depth_files.keys()))
print("Matched image indices:", common_indices)

# Function to create point cloud from a pair
def create_point_cloud(rgb_path, depth_path):
    color = o3d.io.read_image(rgb_path)
    depth = o3d.io.read_image(depth_path)

    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, depth,
        depth_scale=1000.0,   # 1 unit = 1 mm
        depth_trunc=3.0,      # ignore depths > 3 meters
        convert_rgb_to_intensity=False
    )

    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image, intrinsic
    )

    return pcd

# Combine all point clouds
combined_pcd = o3d.geometry.PointCloud()

for idx in common_indices:
    rgb_path = rgb_files[idx]
    depth_path = depth_files[idx]

    print(f"Processing pair: {rgb_path}, {depth_path}")
    pcd = create_point_cloud(rgb_path, depth_path)
    combined_pcd += pcd

# Downsample for faster visualization
combined_pcd = combined_pcd.voxel_down_sample(voxel_size=0.005)

# Save to file
o3d.io.write_point_cloud("desk_combined.ply", combined_pcd)
print("Point cloud saved to desk_combined.ply")


Matched image indices: ['1', '101', '113', '115', '118', '121', '125', '13', '133', '141', '147', '157', '167', '17', '170', '174', '181', '188', '190', '22', '33', '47', '54', '66', '8', '80', '90', '96']
Processing pair: /media/amiyaun/New Volume/rugved/task8/rgb/desk_2_1.png, /media/amiyaun/New Volume/rugved/task8/depth/desk_2_1_depth.png
Processing pair: /media/amiyaun/New Volume/rugved/task8/rgb/desk_2_101.png, /media/amiyaun/New Volume/rugved/task8/depth/desk_2_101_depth.png
Processing pair: /media/amiyaun/New Volume/rugved/task8/rgb/desk_2_113.png, /media/amiyaun/New Volume/rugved/task8/depth/desk_2_113_depth.png
Processing pair: /media/amiyaun/New Volume/rugved/task8/rgb/desk_2_115.png, /media/amiyaun/New Volume/rugved/task8/depth/desk_2_115_depth.png
Processing pair: /media/amiyaun/New Volume/rugved/task8/rgb/desk_2_118.png, /media/amiyaun/New Volume/rugved/task8/depth/desk_2_118_depth.png
Processing pair: /media/amiyaun/New Volume/rugved/task8/rgb/desk_2_121.png, /media/amiya

In [3]:
o3d.visualization.draw([combined_pcd])


[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x599f4aa69130 (threading is enabled)
EGL(1.5)
OpenGL(4.6)
[Open3D INFO] ICE servers: ["stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"]
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.


KeyboardInterrupt: 

In [4]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt

# Load the saved point cloud
pcd = o3d.io.read_point_cloud("desk_combined.ply")

# Estimate normals (optional, useful for more complex processing)
pcd.estimate_normals()

# Run DBSCAN clustering
# eps: distance threshold (try 0.02 ~ 0.05 depending on scene size)
# min_points: minimum cluster size
labels = np.array(pcd.cluster_dbscan(eps=0.02, min_points=20, print_progress=True))

num_clusters = labels.max() + 1
print(f"Detected {num_clusters} clusters")

# Assign random color to each cluster
colors = plt.get_cmap("tab20")(labels / (num_clusters if num_clusters > 0 else 1))
colors[labels < 0] = [0, 0, 0, 1]  # noise (outliers) in black
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])  # RGB only

# Visualize the clustered point cloud
o3d.visualization.draw([pcd])


Precompute neighbors.[========================================] 100%
Detected 650 clusters=============================>] 97%
[Open3D INFO] Window window_1 created.


KeyboardInterrupt: 